In [1]:
from keras.layers import Input, Dense
from keras.models import Model

input_all = Input(shape=(233,), name="input")

x = Dense(512,activation='relu')(input_all)

x = Dense(1024,activation= 'relu')(x)

x = Dense(512)(x)

out = Dense(4)(x)

model = Model(inputs=[input_all], outputs=out)
model.compile(optimizer='rmsprop', loss='mse')
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 233)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               119808    
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 2052      
Total params: 1,171,972
Trainable params: 1,171,972
Non-trainable params: 0
_________________________________________________________________


In [2]:
from emulator import CustomEmulator
import numpy as np

emul = CustomEmulator(500,10)
num_hands = 100000
total_reward = 0
y = .99
e = 0.1

old_state = np.zeros((233))
targetQ = np.zeros((4))
replay_state = []
replay_Q = []
max_replay = 5000

for i in range(num_hands):
    reward_sb = 0
    reward_bb = 0
    #Starts with dealer
    is_sb_turn = True
    
    sb_has_played = False
    bb_has_played = False
  
    emul.new_hand()
    #while hand is still at play
    while(emul.is_round_finished() == False):
           
        emul.make_features()
    
        if(is_sb_turn == True):
            #save old state to update it asap with new rew
            if(sb_has_played == True):
                old_state = sb_features
                targetQ = allQ_sb
                
                ##tochange
            sb_features = np.concatenate([emul.players_cards[1], np.concatenate(
    emul.cards_feature), np.concatenate(emul.actions_feature),np.array([1])]).reshape((1,233))
            
            #run model to choose action
            allQ_sb = model.predict(sb_features)
            action_sb = np.argmax(allQ_sb)
            #print(allQ_sb)
            
            #randomly discover new line
            if(np.random.rand(1) < e):
                action_sb = np.random.randint(0,4)
            
            emul.play_action(action_sb)
                
            
            if(sb_has_played == True):
                targetQ[0,action_sb] = y*np.max(allQ_sb)
                #replay_state.append(old_state.reshape((233,)))
                #replay_Q.append(targetQ.reshape((4,)))

            sb_has_played = True
            is_sb_turn = False
        else:
            if(bb_has_played == True):
                old_state = bb_features
                targetQ = allQ_bb
                
            bb_features = np.concatenate([emul.players_cards[0], np.concatenate(
    emul.cards_feature), np.concatenate(emul.actions_feature),np.array([0])]).reshape((1,233))
            
            allQ_bb = model.predict(bb_features)
            action_bb = np.argmax(allQ_bb)
            
            if(np.random.rand(1) < e):
                action_bb = np.random.randint(0,4)
                
            emul.play_action(action_bb)        
            
            if(bb_has_played == True):
                targetQ[0,action_bb] = y*np.max(allQ_bb)
                #replay_state.append(old_state.reshape((233,)))
                #replay_Q.append(targetQ.reshape((4,)))


            bb_has_played = True
            is_sb_turn = True
     
    r  = emul.get_sb_reward()
    total_reward +=r
    
    if(i%10000 == 0 and i > 10):
         #fit on replay
        model.fit(np.stack(replay_state),np.stack(replay_Q),verbose=1)
    
    if(i%2000== 0):
        print('round',i)
        print('Won(SB):',r,'  Total won (SB):',total_reward)
        print('Replay size:',len(replay_state))
        print(allQ_sb)
        print(emul.get_action_histories_text(hole_cards = True))

    allQ_sb[0,action_sb] = r

    replay_state.append(old_state.reshape((233,)))
    replay_Q.append(targetQ.reshape((4,)))
    
    if(bb_has_played):   
        allQ_bb[0,action_bb] = -r
        replay_state.append(old_state.reshape((233,)))
        replay_Q.append(targetQ.reshape((4,)))
    

    if(len(replay_state) > max_replay):
        replay_state = replay_state[50:]
        replay_Q = replay_Q[50:]

            

round 0
Won(SB): -10   Total won (SB): -10
Replay size: 0
[[-0.09447599 -0.06344476 -0.04876947  0.00524062]]
{'bb_player': ['D8', 'S8'], 'sb_player': ['H4', 'S3']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player folds

round 2000
Won(SB): 20   Total won (SB): -1410
Replay size: 3905
[[-0.04299714  0.04194836  0.01644766 -0.05604092]]
{'bb_player': ['SK', 'CQ'], 'sb_player': ['DQ', 'HK']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player CALL 20
bb_player folds

round 4000
Won(SB): -10   Total won (SB): 4520
Replay size: 4962
[[-0.06532461 -0.03206693 -0.0539715  -0.003291  ]]
{'bb_player': ['C5', 'HA'], 'sb_player': ['H7', 'SK']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player folds

round 6000
Won(SB): 30   Total won (SB): 7150
Replay size: 4955
[[-0.07759221 -0.03331215 -0.03106138  0.01329541]]
{'bb_player': ['DK', 'H2'], 'sb_player': ['H9', 'HQ']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player CALL 20
bb_player raises 

round 26000
Won(SB): 220   Total won (SB): -42540
Replay size: 4953
[[-0.028465   -0.03452818 -0.00485838 -0.01810684]]
{'bb_player': ['D8', 'S6'], 'sb_player': ['DK', 'C4']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player raises to 30
bb_player raises to 40
sb_player raises to 50
bb_player raises to 60
sb_player raises to 70
bb_player raises to 80
sb_player raises to 90
bb_player raises to 100
sb_player raises to 110
bb_player raises to 120
sb_player raises to 130
bb_player raises to 140
sb_player raises to 150
bb_player raises to 160
sb_player raises to 170
bb_player raises to 180
sb_player raises to 190
bb_player raises to 200
sb_player raises to 210
bb_player raises to 220
sb_player raises to 230
bb_player folds

round 28000
Won(SB): 30   Total won (SB): -29110
Replay size: 4986
[[-0.01402878  0.02887073  0.03261352 -0.03678717]]
{'bb_player': ['DT', 'C6'], 'sb_player': ['DA', 'DJ']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player raises to 30
bb_pla

round 58000
Won(SB): -10   Total won (SB): -16020
Replay size: 4969
[[ 0.04490436 -0.0095552   0.03922853 -0.01114222]]
{'bb_player': ['H7', 'D9'], 'sb_player': ['CJ', 'CA']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player folds

Epoch 1/1
4969/4969 [==============================] - 1s 266us/step - loss: 4.8655e-05
round 60000
Won(SB): -10   Total won (SB): -7260
Replay size: 4969
[[ 0.01084906 -0.04797382  0.00874421 -0.02469078]]
{'bb_player': ['CT', 'SK'], 'sb_player': ['CJ', 'D3']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player folds

round 62000
Won(SB): -90   Total won (SB): -10000
Replay size: 4992
[[-0.00782462 -0.03726835 -0.00634965 -0.04476043]]
{'bb_player': ['SQ', 'H2'], 'sb_player': ['C4', 'D9']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player raises to 30
bb_player raises to 40
sb_player raises to 50
bb_player raises to 60
sb_player raises to 70
bb_player raises to 80
sb_player raises to 90
bb_player raises to 500
sb_play

Epoch 1/1
4998/4998 [==============================] - 1s 280us/step - loss: 2.9645e-05
round 90000
Won(SB): -10   Total won (SB): -35560
Replay size: 4998
[[ 0.0299361  -0.01363146  0.02740263 -0.07791464]]
{'bb_player': ['C5', 'SQ'], 'sb_player': ['C7', 'D8']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player folds

round 92000
Won(SB): -50   Total won (SB): -33010
Replay size: 4962
[[ 0.00899094 -0.01759289  0.00894549 -0.05044234]]
{'bb_player': ['CQ', 'D2'], 'sb_player': ['S7', 'H8']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player raises to 30
bb_player raises to 40
sb_player raises to 50
bb_player raises to 60
sb_player folds

round 94000
Won(SB): 20   Total won (SB): -30380
Replay size: 4984
[[ 0.02511916 -0.05197794  0.02748456 -0.0097666 ]]
{'bb_player': ['HQ', 'D4'], 'sb_player': ['C9', 'S9']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player raises to 30
bb_player folds

round 96000
Won(SB): -10   Total won (SB): -24240
Replay siz